In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
# Any results you write to the current directory are saved as output.

# Loading and Preparing Dataset

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
class MNIST_Dataset(Dataset):
    
    def __init__(self, X, y, transform = transforms.Compose(
                     [transforms.ToPILImage(), 
                      transforms.ToTensor(), 
                      transforms.Normalize(mean=(0.5,), std=(0.5,))])):
            
        self.transform = transform
        self.X = X.reshape((-1,56,56)).astype(np.uint8)[:,:,:,None]
        self.y = y
        
    @classmethod
    def from_train(cls, file_path_images, file_path_labels):
        train_images = pd.read_pickle(file_path_images)
        train_labels = pd.read_csv(file_path_labels)
        
        return cls(X=train_images, y=torch.from_numpy(train_labels['Category'].values))
        
    @classmethod
    def from_test(cls, file_path_images):
        test_images = pd.read_pickle(file_path_images)
        
        return cls(X=test_images, y=None)
    
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.transform(self.X[idx])
        # X = X.to(device)
        if self.y is not None:
            return X, self.y[idx]   #.to(device)
        else:
            return X

In [4]:
train_data = pd.read_pickle('images_l.pkl')
train_labels = pd.read_pickle('labels_l.pkl')
test_data = pd.read_pickle('images_test.pkl')
unlabel = pd.read_pickle('images_ul.pkl')

number_train_images = int(train_data.shape[0] * 0.9)

X_train = train_data[:number_train_images]
y_train = train_labels[:number_train_images]

X_validation = train_data[number_train_images:]
y_validation = train_labels[number_train_images:]

In [11]:
batch_size = 32

train_dataset = MNIST_Dataset(X_train, y_train)
validation_dataset = MNIST_Dataset(X_validation, None)
test_dataset = MNIST_Dataset.from_test('images_test.pkl')

# Loading
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset,
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size, 
                                           shuffle=False)

# Neural Network Architecture, and Optimizer, and Loss Functions

In [48]:
class ConvNet_4Layers(nn.Module):
    def __init__(self, input_dim, output_dim, dropout):
        super(ConvNet_4Layers, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=28, kernel_size=2, stride=1),
            nn.ReLU(),
            nn.Conv2d(28, 28, 2),
            nn.BatchNorm2d(28),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(dropout))
        self.layer2 = nn.Sequential(
            nn.Conv2d(28, 56, 2),
            nn.ReLU(),
            nn.Conv2d(56, 56, 2),
            nn.BatchNorm2d(56),
            nn.ReLU(),
            nn.Conv2d(56, 56, 2),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=1),
            nn.Conv2d(56, 56, 3),
            nn.ReLU(),
            nn.BatchNorm2d(56),
            nn.Conv2d(56, 56, 2),
            nn.ReLU(),
            nn.MaxPool2d(2,stride=2),
            nn.Dropout(dropout))
        self.layer3 = nn.Sequential(
            nn.Conv2d(56, 112, 2),
            nn.ReLU(),
            nn.Conv2d(112, 112, 2),
            nn.BatchNorm2d(112),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(dropout))
        self.layer4 = nn.Sequential(
            nn.Conv2d(112, 224, 2),
            nn.ReLU(),
            nn.Conv2d(224, 224, 2),
            nn.BatchNorm2d(224),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(dropout))
        self.fc = nn.Linear(224, output_dim)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

from torchsummary import summary
summary(ConvNet_4Layers(56*56, 36, 0.1), input_size=(1, 56, 56))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 28, 55, 55]             140
              ReLU-2           [-1, 28, 55, 55]               0
            Conv2d-3           [-1, 28, 54, 54]           3,164
       BatchNorm2d-4           [-1, 28, 54, 54]              56
              ReLU-5           [-1, 28, 54, 54]               0
         MaxPool2d-6           [-1, 28, 27, 27]               0
           Dropout-7           [-1, 28, 27, 27]               0
            Conv2d-8           [-1, 56, 26, 26]           6,328
              ReLU-9           [-1, 56, 26, 26]               0
           Conv2d-10           [-1, 56, 25, 25]          12,600
      BatchNorm2d-11           [-1, 56, 25, 25]             112
             ReLU-12           [-1, 56, 25, 25]               0
           Conv2d-13           [-1, 56, 24, 24]          12,600
             ReLU-14           [-1, 56,

In [28]:
def predict(model, data_loader):
    real=[]
    for i, data in enumerate(data_loader):
        output = model(data)
        for i in range(output.shape[0]):
            com = onehot(output[i])
            real.append(com)   
    return torch.Tensor(real)

In [29]:
def onehot(vector):
    digit=vector[0:10]
    alphabet=vector[10:36]
    max_1=torch.argmax(digit,0,keepdim=True)
    max_2=torch.argmax(alphabet,0,keepdim=True)
    one_hot1=torch.FloatTensor(digit.shape)
    one_hot2=torch.FloatTensor(alphabet.shape)
    one_hot1.zero_()
    one_hot2.zero_()
    one_hot1.scatter_(0, max_1, 1)
    one_hot2.scatter_(0, max_2, 1)
    return torch.cat((one_hot1,one_hot2),0).tolist()

## Training Loop

In [41]:
def train_model(model_class, validation_loader, y_validation, dropout=0.5, epochs=30, criterion=nn.CrossEntropyLoss(), learning_rate=1e-3):
    input_dim = 56 * 56
    output_dim = 36
    model = model_class(input_dim, output_dim, dropout)
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    losses_per_epoch = []
    validation_score_per_epoch = []
    for epoch in range(epochs):
        print(epoch)
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs) 
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
       
        validation_predictions = predict(model, validation_loader).numpy()
       
        validation_score_per_epoch.append(accuracy_score(y_validation.tolist(), validation_predictions.tolist()))
        model.train()
        
        losses_per_epoch.append(loss.item())
    print('[Finished Training] Score: %.4f Loss: %.4f' % (validation_score_per_epoch[-1], losses_per_epoch[-1]))
    return model, losses_per_epoch, validation_score_per_epoch

In [42]:
#test
model, loss, score = train_model(ConvNet_4Layers, validation_loader, y_validation, dropout=0.1, epochs=10)

0
1
2
3
4
5
6
7
8
9
[Finished Training] Score: 0.7483 Loss: 2.1303


## Prediction on Test Set 

In [ ]:
test_predictions = predict(model, test_loader)
# print(test_predictions[0])
test_predictions = test_predictions.detach().numpy()

In [ ]:
new_list=[]
for element in test_predictions:
    sub_new_list=''
    for sub_element in element:
        sub_new_list+=str(int(sub_element))
    new_list.append(sub_new_list)
    
d = {'# Id' : [x for x in range(0, len(test_predictions))], 'Category' : new_list}
# print(len(d['Category']))
submission = pd.DataFrame(d)
submission.to_csv(path_or_buf='submission1.csv', index=False)